In [1]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

# Creating subagents

In [2]:
from langchain.tools import tool

@tool
def square_root(x):
    """Calculates the square root of a number"""
    return x**0.5

@tool
def square(x):
    """calculates the square of a number"""
    return x**2

In [3]:
from langchain.agents import create_agent

#create subagents
subagent1=create_agent(model="gpt-5-nano",tools=[square_root])
subagent2=create_agent(model="gpt-5-nano",tools=[square])



# Calling subagents

In [4]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x):
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

main_agent=create_agent(model="gpt-5-nano",
                        tools=[call_subagent_1,call_subagent_2],
                        system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")


# Test

In [5]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='81e33854-461a-4f4c-80f1-8d1960ddd618'),
              AIMessage(content='sqrt(456) = 2 sqrt(114) ≈ 21.354156.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1884, 'prompt_tokens': 202, 'total_tokens': 2086, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1856, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CrnIEzswIng16WXc3ViQzDVF1fvTo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b65a5-75f2-71a3-8d6e-73dca6ba190f-0', usage_metadata={'input_tokens': 202, 'output_tokens': 1884, 'total_tokens': 2086, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_d